<a href="https://colab.research.google.com/github/sbeldine97/Business-Analytics-Report/blob/main/Customer_Offer_Segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Importing the necessary libraries

import json
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
from google.colab import drive
custom_colors = ['#512455', '#FC6700', '#00A1B1', '#939199', '#843A1A']


In [ ]:
# Mounting Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Loading the datasets provided

In [ ]:
#Portfolio dataset
file_path = '/content/drive/MyDrive/Business Analytics Report/portfolio.json'
# Initializing an empty list to store individual JSON objects
json_objects = []
# Reading the JSON file line by line and load each line as a JSON object
with open(file_path, 'r') as json_file:
    for line in json_file:
        try:
            json_data = json.loads(line)
            json_objects.append(json_data)
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON: {e}")

# Converting the list of JSON objects into a DataFrame
portfolio = pd.DataFrame(json_objects)

In [ ]:
#Loading the profile dataset
file_path = '/content/drive/MyDrive/Business Analytics Report/profile.json'
# Initializing an empty list to store individual JSON objects
json_objects = []
# Reading the JSON file line by line and load each line as a JSON object
with open(file_path, 'r') as json_file:
    for line in json_file:
        try:
            json_data = json.loads(line)
            json_objects.append(json_data)
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON: {e}")

# Converting the list of JSON objects into a DataFrame
profile = pd.DataFrame(json_objects)


In [ ]:
#Loading the transactions dataset

file_path = '/content/drive/MyDrive/Business Analytics Report/transcript.json'

# Initializing an empty list to store individual JSON objects
json_objects = []

# Reading the JSON file line by line and load each line as a JSON object
with open(file_path, 'r') as json_file:
    for line in json_file:
        try:
            json_data = json.loads(line)
            json_objects.append(json_data)
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON: {e}")

# Converting the list of JSON objects into a DataFrame
transaction = pd.DataFrame(json_objects)


In [ ]:
portfolio.head()

,reward,channels,difficulty,duration,offer_type,id
0,10,"[email, mobile, social]",10,7.0,bogo,ae264e3637204a6fb9bb56bc8210ddfd
1,10,"[web, email, mobile, social]",10,5.0,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
2,0,"[web, email, mobile]",0,4.0,informational,3f207df678b143eea3cee63160fa8bed
3,5,"[web, email, mobile]",5,7.0,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9
4,5,"[web, email]",20,10.0,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7


In [ ]:
portfolio.head()

,reward,channels,difficulty,duration,offer_type,id
0,10,"[email, mobile, social]",10,7.0,bogo,ae264e3637204a6fb9bb56bc8210ddfd
1,10,"[web, email, mobile, social]",10,5.0,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
2,0,"[web, email, mobile]",0,4.0,informational,3f207df678b143eea3cee63160fa8bed
3,5,"[web, email, mobile]",5,7.0,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9
4,5,"[web, email]",20,10.0,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7


In [ ]:
# Looking into the types of offers available
portfolio['offer_type'].unique()

array(['bogo', 'informational', 'discount'], dtype=object)

In [ ]:
portfolio.shape

(10, 6)

In [ ]:
portfolio.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   reward      10 non-null     int64  
 1   channels    10 non-null     object 
 2   difficulty  10 non-null     int64  
 3   duration    10 non-null     float64
 4   offer_type  10 non-null     object 
 5   id          10 non-null     object 
dtypes: float64(1), int64(2), object(3)
memory usage: 608.0+ bytes


In [ ]:
#Looking into the spread of the numerical data

portfolio.describe()

,reward,difficulty,duration
count,10.000000,10.000000,10.000000
mean,4.200000,7.700000,6.500000
std,3.583915,5.831905,2.321398
min,0.000000,0.000000,3.000000
25%,2.000000,5.000000,5.000000
50%,4.000000,8.500000,7.000000
75%,5.000000,10.000000,7.000000
max,10.000000,20.000000,10.000000


In [ ]:
portfolio = portfolio.join(portfolio.channels.str.join('|').str.get_dummies()).drop('channels', axis=1)

for column in ["email", "mobile", "social", "web"]:
    portfolio = portfolio.rename(columns={column: ("channel_" + column)})

portfolio.head(3)

,reward,difficulty,duration,offer_type,id,channel_email,channel_mobile,channel_social,channel_web
0,10,10,7.0,bogo,ae264e3637204a6fb9bb56bc8210ddfd,1,1,1,0
1,10,10,5.0,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0,1,1,1,1
2,0,0,4.0,informational,3f207df678b143eea3cee63160fa8bed,1,1,0,1


In [ ]:
portfolio['duration_hrs'] = portfolio.duration*24
portfolio

,reward,difficulty,duration,offer_type,id,channel_email,channel_mobile,channel_social,channel_web,duration_hrs
0,10,10,7.0,bogo,ae264e3637204a6fb9bb56bc8210ddfd,1,1,1,0,168.0
1,10,10,5.0,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0,1,1,1,1,120.0
2,0,0,4.0,informational,3f207df678b143eea3cee63160fa8bed,1,1,0,1,96.0
3,5,5,7.0,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9,1,1,0,1,168.0
4,5,20,10.0,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7,1,0,0,1,240.0
5,3,7,7.0,discount,2298d6c36e964ae4a3e7e9706d1fb8c2,1,1,1,1,168.0
6,2,10,10.0,discount,fafdcd668e3743c1bb461111dcafc2a4,1,1,1,1,240.0
7,0,0,3.0,informational,5a8bc65990b245e5a138643cd4eb9837,1,1,1,0,72.0
8,5,5,5.0,bogo,f19421c1d4aa40978ebb69ca19b0e20d,1,1,1,1,120.0
9,2,10,7.0,discount,2906b810c7d4411798c6938adc9daaa5,1,1,0,1,168.0


In [ ]:
portfolio.dtypes

reward              int64
difficulty          int64
duration          float64
offer_type         object
id                 object
channel_email       int64
channel_mobile      int64
channel_social      int64
channel_web         int64
duration_hrs      float64
dtype: object

In [ ]:
portfolio.isna().sum()

reward            0
difficulty        0
duration          0
offer_type        0
id                0
channel_email     0
channel_mobile    0
channel_social    0
channel_web       0
duration_hrs      0
dtype: int64

In [ ]:
profile.head()

,gender,age,id,became_member_on,income
0,None,118,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN
1,F,55,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0
2,None,118,38fe809add3b4fcf9315a9694bb96ff5,20180712,NaN
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0
4,None,118,a03223e636434f42ac4c3df47e8bac43,20170804,NaN


In [ ]:
profile.shape

(17000, 5)

In [ ]:
profile.dtypes

gender               object
age                   int64
id                   object
became_member_on     object
income              float64
dtype: object

In [ ]:
# Convert `became_member_on` in user data to datetime
profile['income'] = profile.income.astype(int) # `income
profile['membership_date'] = pd.to_datetime(profile.became_member_on.astype(str))
profile.drop('became_member_on', axis=1, inplace=True)
profile.head()

IntCastingNaNError: ignored

In [ ]:
profile_df.isna().sum()

gender             2175
age                   0
id                    0
income             2175
membership_date       0
dtype: int64

In [ ]:
profile_df.describe()

In [ ]:
transcation_df=pd.read_excel("/content/drive/MyDrive/Airtribe/Transcation.xlsx")
transcation_df.head()

In [ ]:
transcation_df.shape

In [ ]:
transcation_df.dtypes

In [ ]:
transcation_df.dtypes


In [ ]:
transcation_df.isna().sum()

In [ ]:
print('Overall event count:')
print(transcation_df.event.value_counts())


In [ ]:
# Fill missing values with 0
transcation_df.fillna(0, inplace=True)
transcation_df.isnull().sum().sum()

In [ ]:
transcation_df.head()


There is no problem with the missing values here:
For "offer received" and "offer viewed" events: no transaction was made so there shouldn't be an amount and no reward was given so there shouldn't be a reward
For "offer completed" events: no transaction was made so there shouldn't be an amount
For "transaction" events: there is no offer corresponding to transactions so there shouldn't be an offer_id or a reward
Since these values aren't actually missing values, we will be filling all of them with 0.

In [ ]:
transcation_df.describe()

In [ ]:
print('Duplicated users:', profile_df.duplicated().sum())
print('Duplicated events:', transcation_df.duplicated().sum())

In [ ]:
# Duplicated events
dup_events = transcation_df[transcation_df.duplicated(keep=False)]
dup_events.head()

In [ ]:
# Drop duplicate events
transcation_df.drop_duplicates(inplace=True)
transcation_df.duplicated().sum()

In [ ]:
transcation_df.shape

In [ ]:
# Sort portfolio_df
portfolio_df = portfolio_df.sort_values(['offer_type', 'difficulty']).reset_index(drop = True)
portfolio_df.head(10)

In [ ]:
# Add the offer alias
from string import ascii_uppercase
portfolio_df['offer_alias']=[ascii_uppercase[i] for i in range(portfolio_df.shape[0])]
portfolio_df

In [ ]:

# declaring data
data = profile_df['gender'].value_counts()
keys = ['Male', 'Female', 'Other']

plt.figure(figsize = (5,5))
# define Seaborn color palette to use
palette_color = sns.color_palette('dark')

# plotting data on chart
plt.pie(data, labels=keys, colors=palette_color, autopct='%1.1f%%')
plt.title('Gender', fontsize = 16)
# displaying chart
plt.show()

In [ ]:
fig, axs = plt.subplots(1,2, figsize = (12,4))
sns.boxplot(data = profile_df, x = 'age', ax = axs[0])
sns.histplot(data = profile_df, x = 'age', ax = axs[1])
axs[0].xaxis.label.set_size(15)
axs[1].xaxis.label.set_size(15)
axs[1].yaxis.label.set_size(15)


In [ ]:
print(profile_df.age.max())
print(profile_df.age.min())
profile_df.shape

In [ ]:
age_groups=pd.cut(profile_df['age'],
      bins = [profile_df['age'].min(), 26, 36, 46, 56, 66, 76, 86, profile_df['age'].max()],
      labels = ['18-25ys', '26-35ys', '36-45ys', '46-55ys', '56-65ys', '66-75ys', '76-85ys', '> 86ys'])

In [ ]:
age_group_percentage = (age_groups.value_counts().sort_index() / profile_df.shape[0] * 100).round(1)

In [ ]:
plt.figure(figsize = (8, 5))
yticklabels = [str(y) + '%' for y in np.arange(0,25,5)]
plt.yticks(np.arange(0,25,5), labels = yticklabels)
plt.xticks(rotation = 45, fontsize = 12)
plt.xlabel('Age', fontsize = 15)
plt.ylabel('Percentage %', fontsize = 15)
plt.title('Age Distribution', fontsize = 18)

bar_plot = plt.bar(x = age_group_percentage.index.values, height = age_group_percentage)
for i,bar in enumerate(bar_plot.patches):
    x, y = bar.get_xy()
    plt.text(x+bar.get_width()/2, y+bar.get_height()+0.2,
            str(age_group_percentage[i]) + '%',
            ha = 'center', weight = 'bold')

plt.tight_layout()
plt.show()

## Income

In [ ]:
profile_df.columns

In [ ]:
fig, axs = plt.subplots(1,2, figsize = (12,4))
sns.boxplot(data = profile_df, x = 'income', ax = axs[0])
sns.histplot(data = profile_df, x = 'income', ax = axs[1])
axs[0].xaxis.label.set_size(15)
axs[1].xaxis.label.set_size(15)
axs[1].yaxis.label.set_size(15)


In [ ]:
print(profile_df.income.min())
print(profile_df.income.max())
profile_df.income.value_counts().reset_index()

In [ ]:
income_group=pd.cut(profile_df['income'],
      bins = [profile_df['income'].min(), 45000, 60000, 75000,90000, 105000, profile_df['income'].max()],
      labels = ['30-45k', '45-60k', '60-75k', '75-90k', '90-105k','>105k'])
# Visualize customers by age groups
income_group_percentage = (income_group.value_counts().sort_index() / profile_df.shape[0] * 100).round(1)

In [ ]:

plt.figure(figsize = (8, 5))
yticklabels = [str(y) + '%' for y in np.arange(0,30,5)]
plt.yticks(np.arange(0,30,5), labels = yticklabels)
plt.xticks(rotation = 45, fontsize = 12)
plt.xlabel('Income', fontsize = 15)
plt.ylabel('Percentage %', fontsize = 15)
plt.title('Income Distribution', fontsize = 18)

bar_plot = plt.bar(x = income_group_percentage.index.values, height = income_group_percentage)
for i,bar in enumerate(bar_plot.patches):
    x, y = bar.get_xy()
    plt.text(x+bar.get_width()/2, y+bar.get_height()+0.2,
            str(income_group_percentage[i]) + '%',
            ha = 'center', weight = 'bold')

plt.tight_layout()
plt.show()

In [ ]:
sns.histplot(data = profile_df, x = 'age', hue = 'gender', fill = False)
plt.title('Age vs Gender', fontsize = 16)
plt.show()

In [ ]:
sns.histplot(data = profile_df, x = 'income', hue = 'gender', fill = False)
plt.title('Income vs Gender', fontsize = 16)
plt.show()

In [ ]:
# Extract the year,month,week from the became_member_on feature

profile_df['year'] = profile_df['signup_date'].dt.year
profile_df['month'] = profile_df['signup_date'].dt.month
profile_df['day']=profile_df['signup_date'].dt.day_name()
profile_df['member_since_how_manydays'] = (pd.to_datetime('today') - profile_df['signup_date']).astype('timedelta64[D]').astype(int)



In [ ]:
profile_df.head()

##Member growth by year

In [ ]:
sns.countplot(data = profile_df, x = 'year')
plt.title('Number of New Members by Year', fontsize = 16)
plt.xticks(fontsize = 12); plt.yticks(fontsize = 12)
plt.xlabel('Year', fontsize = 14); plt.ylabel('Count', fontsize = 14)
plt.show()

In [ ]:
plt.figure(figsize = (10,5))
sns.countplot(data = profile_df.sort_values('month'), x = 'month')
plt.title('Number of New Members by Month', fontsize = 16)
plt.xticks(fontsize = 12); plt.yticks(fontsize = 12)
plt.xlabel('Month', fontsize = 14); plt.ylabel('Count', fontsize = 14)
plt.show()

In [ ]:
plt.figure(figsize = (10,5))
sns.countplot(data = profile_df.sort_values('day'), x = 'day')
plt.title('Number of New Members by Weekday', fontsize = 16)
plt.xticks(fontsize = 12); plt.yticks(fontsize = 12)
plt.xlabel('Weekday', fontsize = 14); plt.ylabel('Count', fontsize = 14)
plt.show()

In [ ]:
p = portfolio_df[["email", "mobile","social","web"]].groupby(["email", "mobile","social","web"]).size()
print(p)
plt.figure(figsize = (12,5))
sns.barplot(x = p.index.values, y = p.values)
plt.xticks(rotation = None, fontsize = 12)
plt.yticks(np.arange(5), fontsize = 14)
plt.xlabel('Channels', fontsize = 16, fontweight = 'bold')
plt.ylabel('Count', fontsize = 16, fontweight = 'bold')
plt.title('Promotion Channel', fontsize = 20)
plt.tight_layout()
plt.show()

In [ ]:
transcation_df['event'].value_counts()

In [ ]:
fig = plt.figure(figsize = (7,5))
transcation_df['event'].value_counts().plot(kind = 'bar')
plt.xticks(rotation = None)
plt.ylabel('Count')
plt.title('Event counts', fontsize = 16)
plt.show()

In [ ]:
transcation_df.head()

In [ ]:
transcation_df.columns

In [ ]:
profile_df.head()

In [ ]:
# Add column `days_since_start`
transcation_df = transcation_df.assign(days_since_start = transcation_df['hours_till_action']/24).astype({'days_since_start': 'int64'})
print('The minimum and the maximum of days_since_start: {} and {}'.format(transcation_df['days_since_start'].min(), transcation_df['days_since_start'].max()))


In [ ]:
plt.hist(transcation_df['days_since_start'], bins = 20)
plt.xlabel('Days', fontsize = 14)
plt.ylabel('Events Count', fontsize = 14)
plt.title('Events Occurrence over time', fontsize = 1)
plt.show()

In [ ]:
transcation_df[transcation_df['event'] == 'offer received'].groupby('hours_till_action').count()

In [ ]:
transcation_df[transcation_df['event'] == 'offer completed'].groupby('hours_till_action').count()

In [ ]:
# Find how many events occurred in each hour
viewed_hourly = transcation_df[transcation_df['event'] == 'offer viewed'].value_counts('hours_till_action').sort_index()
completed_hourly = transcation_df[transcation_df['event'] == 'offer completed'].value_counts('hours_till_action').sort_index()
transaction_hourly = transcation_df[transcation_df['event'] == 'transaction'].value_counts('hours_till_action').sort_index()
print(len(viewed_hourly), len(completed_hourly), len(transaction_hourly))

In [ ]:
# Visualize
hours = viewed_hourly.index.values
plt.figure(figsize=(14, 5))
# axs[0].stem(received_hourly.index.values, received_hourly, linefmt = ':k')
plt.plot(hours, viewed_hourly, lw = 3)
plt.plot(hours, completed_hourly, lw = 3)
plt.plot(hours, transaction_hourly, lw = 3)
plt.legend(['Offer Viewed', 'Offer Completed', 'Transactions'], loc = 'upper right', ncol = 1, fontsize = 10)
plt.xlabel('Hours', fontsize = 16)
plt.ylabel('Events Count', fontsize = 16)
plt.title('Events Occurrence Hourly', fontsize = 18)

In [ ]:
# Find how many events occurred from 0 to 20 days
viewed_daily = transcation_df[transcation_df['event'] == 'offer viewed'].value_counts('days_since_start').sort_index()
completed_daily = transcation_df[transcation_df['event'] == 'offer completed'].value_counts('days_since_start').sort_index()
transaction_daily = transcation_df[transcation_df['event'] == 'transaction'].value_counts('days_since_start').sort_index()
print(len(viewed_daily), len(completed_daily), len(transaction_daily))

In [ ]:
# Visualize
days = viewed_daily.index.values
plt.figure(figsize=(14, 5))
# axs[0].stem(received_hourly.index.values, received_hourly, linefmt = ':k')
plt.plot(days, viewed_daily, lw = 3)
plt.plot(days, completed_daily, lw = 3)
plt.plot(days, transaction_daily, lw = 3)
plt.legend(['Offer Viewed', 'Offer Completed', 'Transactions'], loc = 'upper right', ncol = 1, fontsize = 10)
plt.xlabel('Days', fontsize = 16)
plt.ylabel('Events Count', fontsize = 16)
plt.title('Events Occurrence Daily', fontsize = 18)

In [ ]:
transcation_df.columns

In [ ]:
portfolio_df.columns

**which Offer were Popular**

In [ ]:
# Create a df that only contains events about offers
df_event = transcation_df[~(transcation_df['event'] == 'transaction')]

# Merge df_event with and offers
event_offer = pd.merge(df_event,portfolio_df, left_on = 'offer_id', right_on = 'offer_id').drop('offer_id', axis = 1)

# Reorder columns of the merged df
col_names = ['cust_id', 'event', 'offer_alias', 'reward',
       'email','mobile','social','web', 'difficulty', 'Duration', 'offer_type', 'hours_till_action', 'days_since_start']
event_offer = event_offer[col_names]


In [ ]:
event_offer.sample(3)

In [ ]:
# Find out the number of time being received for each offer
offer_received = event_offer[event_offer['event'] == 'offer received']
p0 = offer_received.value_counts('offer_alias').sort_values(ascending = False)

# Find out the number of time being completed for each offer
offer_completed = event_offer[event_offer['event'] == 'offer completed']
p2 = offer_completed.value_counts('offer_alias').sort_values(ascending = False)

In [ ]:
print(p0,p2)

In [ ]:
# Overall completion rate of each offer
overall_complete_rate = (p2.sort_index() / p0.sort_index()).sort_values(ascending = False)

# Visualize
plt.figure(figsize=(10,5))
plt.bar(x = overall_complete_rate.index.values, height = overall_complete_rate.values)
plt.title('Overall Complete Rate %', fontsize = 18)
plt.xlabel('Offer', fontsize = 15)
plt.ylabel('Percentage', fontsize = 15)
plt.yticks(np.arange(0,1.2, 0.2), labels = ['0%', '20%', '40%', '60%', '80%', '100%'])
plt.show()

In this section we tackle the following:

Validate the viewed, completed and recieved offers and create customer transactions and stats.

Create RFM Clusters.

Handle missing values for age, gender and income.

Handle categorical variables.

Remove unnecessary columns for the analysis

Handling the offers events and the transactions
In this section. we need to validate the transactions that occurred as a result of viewing the offer or it is an independent event.

As have been discussed before in the data understanding section regarding the offer completion. We need to confirm for each user if the user has viewed and completed the offer within the validity period while handling the issue of missing offer_id with the transaction event and the missing offer complete event with the informational offers.

The offers is considered completed:

BOGO and discount offers: offer_time_recieved < offer_time_viewed < transaction_time < offer_time_completed < offer_end_time (within the time validaity)

Informational offers: offer_time_recieved < offer_time_viewed < transaction_time < offer_end_time

After finding the effective offers. we can split the customers into 4 groups:

1) people who responded to offers and completed an offer (TR)

2) people who viewed an offer but did not complete the offer(TN)

3) people who have not recieved an offer or have not viewed the offer and have not made a completed the offer(CN)

4) people who have not recieved an offer or have not viewed the offer but have completed the offer(CR)

In [ ]:
# merge the offer with offer portofolio
transcript_df = transcation_df.merge(portfolio_df, on ='offer_id', how='left')
transcript_df.head()

In [ ]:
transcript_df[transcript_df.event !='transaction'].sort_values(['cust_id', 'hours_till_action'])

In [ ]:
transcript_df[(transcript_df.event !='transaction') & (transcript_df.cust_id=='0009655768c64bdeb2e877511632db8f')].sort_values(['cust_id', 'hours_till_action'])

In [ ]:
all_offers = transcript_df.sort_values(['cust_id', 'hours_till_action'])
all_offers.head()

In [ ]:
def split_transcation_on_event(transcript_df):
    """
        This Method splits the transcripts dataframe into 4 dataframes based on the event type
        Args:
            transcation_df(pd.DataFrame): Dataframe for the transcations
        Returns:
            transactions(pd.DataFrame): contains the transaction events
            viewed_offers(pd.DataFrame): contains the viewed events
            recieved_offers(pd.DataFrame): contains the recieved events
            completed_offers(pd.DataFrame): contains the completed events
    """
    received = transcript_df[transcript_df.event == 'offer received']
    completed = transcript_df[transcript_df.event == 'offer completed']
    viewed = transcript_df[transcript_df.event == 'offer viewed']
    transactions = transcript_df[transcript_df.event == 'transaction']

    return received, completed, viewed, transactions

In [ ]:
received, completed, viewed, transactions = split_transcation_on_event(transcript_df)

In [ ]:
received.head()

In [ ]:
def create_split_offers_df(transcript_df):
    """

        split the offers, add and remove unnecessary columns from dataframes
    """
    received, completed, viewed, transactions = split_transcation_on_event(transcript_df)

    received.rename(columns={'hours_till_action':'time_received'}, inplace=True)

    received.drop(columns=['event', 'amount','days_since_start'], inplace=True, axis =1)

    # add expected finish time to the received data frame
    # duration is in days while time is in hours
    received['expected_finish'] = received.time_received + received.Duration * 24

    drop_cols = ['event', 'difficulty', 'Duration', 'offer_type', 'reward',\
                'email', 'mobile', 'social', 'web', 'amount','days_since_start']
    viewed.rename(columns={'hours_till_action':'time_viewed'}, inplace=True)
    viewed.drop(columns=drop_cols, inplace=True, axis =1)


    drop_cols = ['event', 'difficulty', 'Duration', 'offer_type', 'reward',\
                'email', 'mobile', 'social', 'web', 'amount','days_since_start']
    completed.rename(columns={'hours_till_action':'time_completed'}, inplace=True)
    completed.drop(columns=drop_cols, inplace=True, axis =1)
    drop_cols = ['event', 'difficulty', 'Duration', 'offer_type', 'reward',\
                'email', 'mobile', 'social', 'web','offer_id','days_since_start']
    transactions.rename(columns={'hours_till_action':'transaction_time'}, inplace=True)
    transactions.drop(columns=drop_cols, inplace=True, axis =1)

    return received, completed, viewed, transactions

In [ ]:
received, completed, viewed, transactions=create_split_offers_df(transcript_df)
received.head()

In [ ]:
def completed_set(recieved_row, completed):
    """
        This method returns completed and the offer completion time
        Args:
            recieved_row: dataframe containing received events
            completed: dataframe containing completed events
        Returns:
            completed_flag : 1 if the offer was completed on time
            expected_finish: offer completion time

    """
    completed_flag = 0
    finish_time_flag = recieved_row.expected_finish

    completed_set = completed[(completed.offer_id == recieved_row.offer_id) &
                               (completed.time_completed >= recieved_row.time_received) &
                               (completed.time_completed <= recieved_row.expected_finish)]
    if(len(completed_set)>0):
        completed_flag = 1
        finish_time_flag = completed_set.time_completed.iloc[0]
    return pd.Series([completed_flag, finish_time_flag], index=['completed','finish'])

In [ ]:
def viewed_set(row_received,viewed):
    """
        This method returns viewed, offer view time and success
        A transaction is considered successful if it was completed and viewed within the offer duration
        if it was viewed before rhe completion time (finish) that mean it was completed before being viewed
        Args:
            recieved_row: dataframe containing completed events
            completed: dataframe containing completed events
        Returns:
            viewed : 1 if the offer was viewed
            view_time: offer view time
            success: succesful offer
    """
    viewed_flag = 0
    view_time = np.nan
    success = 0

    viewed_set = viewed[(viewed.offer_id == row_received.offer_id) &
                   (viewed.time_viewed >= row_received.time_received) &
                   (viewed.time_viewed <= row_received.finish)]
    if(len(viewed_set)>0):
        viewed_flag = 1
        view_time = viewed_set.time_viewed.iloc[0]
        success = 1 if row_received.completed else 0

    return pd.Series([viewed_flag, success, view_time], index=['viewed','success', 'view_time'])

In [ ]:
def handle_completed_recieved(received, completed):
    """
        This Method takes the recieved offers and the compeleted offers
        then returns completed an finish time
        Args:
            recieved: dataframe containing received events
            completed: dataframe containing completed events
        Returns:
            recieved dataframe updated with the completed and offer finish time
    """

    recieved_updated = received.copy()
    recieved_updated[['completed','finish']] = recieved_updated.apply(lambda row: completed_set(row, completed), axis=1)
    return recieved_updated

In [ ]:
def handle_viewed_completed(received_modified, viewed):
    """
        This Method takes the updated recieved dataframe and the viewed offers
        then returns successful offers, viewed, and offer view time
        Args:
            recieved: dataframe containing received events
            completed: dataframe containing completed events
        Returns:
            recieved dataframe updated with the completed and offer finish time
    """
    viewed_updated = received_modified.copy()
    viewed_updated[['viewed','success', 'view_time']] = viewed_updated.apply(lambda row: viewed_set(row, viewed), axis=1)

    return viewed_updated

In [ ]:
def handle_completed_not_viewed(viewed):
    """
        This Method takes the updated recieved dataframe
        then returns completed_not viewed offers
        (offers that have been completed but are not viewed nor successful)
        Args:
            recieved: dataframe containing received events
            completed: dataframe containing completed events
        Returns:
            recieved dataframe updated with the completed and offer finish time
    """
    viewed_modified = viewed.copy()
    viewed_modified['completed_not_viewed'] = 0
    condition = (viewed_modified.completed==1) & (viewed_modified.viewed==0) & (viewed_modified.success==0)
    viewed_modified.loc[condition,'completed_not_viewed'] = 1

    return viewed_modified

def get_successful_informational_transactions(row, transactions):
    """
      This Method takes the updated merged recieved and the viewed for informational offers
        then find completed offers from these
        Args:
            row: merged received and viewed offers
            transactions: dataframe containing transactions
        Returns:
            success if the transaction was compeleted during the promotion period
    """
    transaction_condition = (transactions.transaction_time >=row.time_received) & ((transactions.transaction_time <=row.expected_finish))
    success = 1 if len(transactions[transaction_condition] ) > 0 else 0
    return pd.Series([success],index =['success'] )

def handle_informational_offers(transactions, data):
    """
    This function handles the case of informational offers since it does not have offer completed event
    viewed offers and transaction within the
    Args:
            transactions customer transactions dataframe
            received offers dataframe
            viewed offers dataframe
    Returns:
        viewed , success
    """


    # we want informational offers that conform with the following condition
    # time_received <= time_viewed <= offer duration
    data_modified = data.copy()
    validation_condition = (data_modified.viewed == 1)&\
                           (data_modified.offer_type=='informational')

    # get the transactions within the condition time
    data_modified.loc[validation_condition,'success'] = data_modified[validation_condition].apply(lambda row: get_successful_informational_transactions(row, transactions), axis=1)

    data_modified.loc[validation_condition,'completed'] = data_modified.loc[validation_condition,'success']

    return data_modified

def get_transaction_stats(row, transactions):
    """
        This function gets the summary for each offer
        - sums all amount of transactions during the offer time
        - get the offer_profit of offer
        - non offer transactions sum
        - assume for non offer transaction that the amount of profit ==non offer sum
    """
    total_spent_offer = 0
    total_spent_non_offer = 0
    profit = 0
    # non offer profit is the amount spent
    # if the offer is success the spent sum transactions from view_time till expected finish
    if row.viewed == 1:
        transactions_offer = transactions[(transactions.transaction_time>= row.view_time ) & (transactions.transaction_time<= row.expected_finish )]
        total_spent_offer =  transactions_offer.amount.sum()
        transactions_non_offer = transactions[(transactions.transaction_time < row.view_time ) & (transactions.transaction_time >= row.time_received )]
        total_spent_non_offer =  transactions_non_offer.amount.sum()
        cost = row.reward if row.completed == 1 else 0
        profit = total_spent_offer - cost
    else:
        transactions_non_offer = transactions[(transactions.transaction_time>= row.time_received ) & (transactions.transaction_time<= row.expected_finish )]
        total_spent_non_offer =  transactions_non_offer.amount.sum()
        cost = row.reward if row.completed == 1 else 0
        profit = total_spent_non_offer - cost

    total_spent = total_spent_non_offer + total_spent_offer
    return pd.Series([total_spent_offer, total_spent_non_offer, profit, total_spent],index =['total_spent_offer','total_spent_non_offer','profit', 'total_spent'] )

def enhance_transaction_df(transactions, offers):
    """
        This function calculates the customer amount spent during offer, profit_offer, non_offer perios

    """
    transaction_updated = offers.copy()
    transaction_updated[['total_spent_offer','total_spent_non_offer','profit', 'total_spent']] = transaction_updated.apply(lambda row: get_transaction_stats(row, transactions), axis=1)
    return transaction_updated

def get_has_profit(row):
    """
        This method returns if this transaction has profit or no
    """
    has_profit = 1 if (( row.viewed == 1 and row.profit > 0) or (row.viewed == 0 and row.total_spent_non_offer > 0)) else 0
    return pd.Series([has_profit],index =['has_profit'] )

def get_completed_offer(row):
    """
        This method returns if this offer was completed or not
    """
    completed_offer = 1 if (( row.completed == 1 ) or (row.completed_not_viewed == 1)) else 0
    return pd.Series([completed_offer],index =['completed_offer'] )

def get_user_stats(offers):
    """
        This function gets the summary for each offer
        - get the number of bogo completed, num of completed offers, num of compelted discounts
        - has_profit if the profit more than 0 for non offer if it is more than 1 then it an offer
    """
    offers_modified = offers.copy()
    offers_modified['num_bogo_completed'] = [offers_modified[offers_modified.offer_type=='bogo']['completed'].sum()] * len(offers_modified)
    offers_modified['num_discount_completed'] = [offers_modified[offers_modified.offer_type=='discount']['completed'].sum()] * len(offers_modified)
    offers_modified['num_informational_completed'] = [offers_modified[offers_modified.offer_type=='informational']['completed'].sum()] * len(offers_modified)
    offers_modified['successful_offers'] = [offers_modified['success'].sum()] * len(offers_modified)
    offers_modified['non_successful_offers'] = [offers_modified[offers_modified.success == 0]['success'].count()] * len(offers_modified)
    offers_modified['completed_offer'] = offers_modified.apply(lambda row: get_completed_offer(row), axis=1)
    return offers_modified

def get_user_group(offers):
    """
        This functions divides the customers into TR, TN, CN, CR. the group we will be assumed as for
        TR: customers who viewed and completed an offer (success = 1)
        TN: customers who viewed the offer did not complete it
        CR: Customers who compelted the offer without viewing the offer (will assume those who have spent more before viewing the offer in this group)
        CN: Customers who did not compelted the offer nor viewed it.
    """
    offers_modified = offers.copy()
    TR_CONDITION = (offers_modified.success == 1.0 ) | ((offers_modified.viewed == 1.0) & (offers_modified.completed == 1.0)) & (offers_modified.total_spent_offer>offers_modified.total_spent_non_offer)
    TN_CONDITION = ((offers_modified.viewed == 1.0) & (offers_modified.completed == 0))
    CR_CONDITION = (offers_modified.completed_not_viewed == 1) | ((offers_modified.viewed == 1.0) & (offers_modified.completed == 1.0) & (offers_modified.total_spent_offer<offers_modified.total_spent_non_offer))
    offers_modified['segment'] = ['CN'] * len(offers_modified)
    offers_modified.loc[TR_CONDITION,'segment'] =  'TR'
    offers_modified.loc[TN_CONDITION,'segment'] =  'TN'
    offers_modified.loc[CR_CONDITION,'segment'] =  'CR'
    return offers_modified

In [ ]:
def offer_analysis(transcript_df):
    """
    This functions looks into offers and create the following columns
    viewed_offer, sucessful_offer(recieved,viewed and completed), tried_offer(received, viewed & not completed),

    """
    received, completed, viewed, transactions = create_split_offers_df(transcript_df)
    # skip when the user has only transactions:
    customer_offers_modified = received
    if len(received) > 0:
            # handle the recieved completed offers completed, finish
            customer_offers_modified = handle_completed_recieved(received, completed)
            # handle viewed & completed --> sucess and we get finish time [if success TR group 1 else TN]
            customer_offers_modified = handle_viewed_completed(customer_offers_modified, viewed)

            # handle informational offers for completed offers
            customer_offers_modified = handle_informational_offers(transactions, customer_offers_modified)

            # handle not viewed & completed  --> does not need a special handle it is accounted using existening columns
            # this might be redundet (viewed & complete but not successful which means not an offer)
            customer_offers_modified = handle_completed_not_viewed(customer_offers_modified)

            # get the transactions amount and offer id
            customer_offers_modified = enhance_transaction_df(transactions, customer_offers_modified)

            #[num of offers completed for each offer id, num of offers viewed depending on type, num of all offers completed ]
            customer_offers_modified = get_user_stats(customer_offers_modified)

            #segment groups
            customer_offers_modified = get_user_group(customer_offers_modified)

            return customer_offers_modified

    else:
        return None


def get_offer_df(all_offers):
    """
        This function creates the offer dataframe for all customers
    """
#     tqdm.pandas()
    return all_offers.groupby('cust_id').apply(offer_analysis).reset_index(drop=True)

In [ ]:
#get all the customer offers dataframe
customers_transactions_df = get_offer_df(all_offers)
customers_transactions_df.head()

In [ ]:
customers_transactions_df.shape

In [ ]:
customers_transactions_df.info()

In [ ]:
customers_transactions_df.completed_offer.value_counts()

In [ ]:
customers_transactions_df.segment.value_counts()

In [ ]:
customers_transactions_df.to_csv('customers_transactions.csv', index=False)

Create RFM Clusters
RFM is an abbreviation for Recency, Frequency, and Monetary Value. It is a method used for customer clustering(segementation) techinque that is used for targeted marketing. This method helps in targeted marketing by examining how recently did the customer purchase(recency), how often they purchase(frequency), and how much they spend(monetary). These attributes will provide customer segements as follows:

- Low value: customers who are the less active, not very frequent buyer and generates very low or negative
revenue.

- Mid value: in between customers

- High value: customers who have bought recently, frequntly and spend the most compared to other customers.
So the steps to create these clusters are as follows:

- Step 1: calculate the recency, frequency and monetary value for each customer.

- Step 2: Create cluster for each of the attributes (recency, frequency and monetary).

- Step 3: Create the combined RFM clusters.
Recency

In [ ]:
transactions = transcation_df[transcation_df.event == 'transaction']

In [ ]:
transactions.head()

In [ ]:
transactions.shape

##**Recency**

In [ ]:
def calculate_recency(transactions):
    '''
        This method calculates a customer recency score based on purchase and transaction history
        Args:
            transaction(pd.DataFrame): dataframe containing the customers transactions
        Returns:
            Recency dataframe for each customer
    '''
    #get the last transaction
    max_purchase = transactions.groupby('cust_id').hours_till_action.max().reset_index()
    max_purchase.columns = ['cust_id','max_purchase_date']

    max_purchase['recency'] = (max_purchase['max_purchase_date'].max() - max_purchase['max_purchase_date'])

    return max_purchase[['cust_id','recency']]

def create_clusters(clusters_num, df, target_field_name, ascending=False):
    """
        This method creates clusters for the given attribute
        Args:
            df(pd.DataFrame): dataframe
            clusters_num: number of clusters
            target_field_name: clusters sorting field
            ascending: ordering
        Returns:
            a dataframe with the clusters numbers added to the dataframe
    """
    kmeans = KMeans(n_clusters=4)
    kmeans.fit(df[[target_field_name]])
    df['{}_cluster'.format(target_field_name)] = kmeans.predict(df[[target_field_name]])
    # order the created clustered
    df_final = order_clusters('{}_cluster'.format(target_field_name), target_field_name, df, ascending)
    return df_final

def order_clusters(cluster_field_name, target_field_name, df, ascending=False):
    """
         This method orders the cluster numbers based on field
        Args:
            transaction(pd.DataFrame): dataframe containing the customers transactions
            cluster_field_name : field of interest for segmentation
            target_field_name: clusters sorting field
        Returns:
            Recency dataframe for each customer
    """
    new_cluster_field_name = 'new_' + cluster_field_name
    df_new = df.groupby(cluster_field_name)[target_field_name].mean().reset_index()
    df_new = df_new.sort_values(by=target_field_name,ascending=ascending).reset_index(drop=True)
    df_new['index'] = df_new.index
    df_final = pd.merge(df,df_new[[cluster_field_name,'index']], on=cluster_field_name)
    df_final = df_final.drop([cluster_field_name],axis=1)
    df_final = df_final.rename(columns={"index":cluster_field_name})
    return df_final

In [ ]:
customer_recency = calculate_recency(transactions)
customer_recency.head()

In [ ]:
customer_recency.shape

In [ ]:
customer_recency.recency.hist(figsize=(10,5));
plt.title('Customer Recency Distribution');
plt.xlabel('Recency Value');
plt.ylabel('Customers Count');

In [ ]:
clusterd_customer_recency = create_clusters(clusters_num=4, target_field_name='recency', df=customer_recency,ascending=False)
clusterd_customer_recency.head()

In [ ]:
clusterd_customer_recency.groupby('recency_cluster')['recency'].describe()

##**Frequency**

In [ ]:
def calculate_frequency(transactions):
    '''
        This method calculates a customer frequency score based on the number of transactions
        Args:
            transaction(pd.DataFrame): dataframe containing the customers transactions
        Returns:
            Frequency dataframe for each customer
    '''
    #get the last transaction
    tx_frequency = transactions.groupby('cust_id').amount.count().reset_index()
    tx_frequency.columns = ['cust_id','frequency']

    return tx_frequency

In [ ]:
customer_fq = calculate_frequency(transactions)
customer_fq.head()

In [ ]:
# determine the frequency for a particular cust_id
customer_fq.loc[customer_fq['cust_id'] == '00116118485d4dfda04fdbaba9a87b5c']

In [ ]:
customer_fq.frequency.hist(figsize=(10,5))
plt.title('Customer Frequency Distribution');
plt.xlabel('Frequency Value');
plt.ylabel('Customers Count');

In [ ]:
# create ordered clusters of the customer frequency
clusterd_customer_fq = create_clusters(clusters_num=4, target_field_name='frequency', df=customer_fq,ascending=True)
clusterd_customer_fq.head()

In [ ]:
# determine the frequency for a particular cust_id
clusterd_customer_fq.loc[clusterd_customer_fq['cust_id'] == '00116118485d4dfda04fdbaba9a87b5c']

In [ ]:
clusterd_customer_fq.frequency_cluster.value_counts()

##**Monetary**

In [ ]:
def calculate_revenue(transactions):
    '''
        This method calculates a customer revenue score based on the amount of transactions
        Args:
            transaction(pd.DataFrame): dataframe containing the customers transactions
        Returns:
            Revenue dataframe for each customer
    '''
    #get the last transaction
    tx_revenue = transactions.groupby('cust_id').amount.sum().reset_index()
    tx_revenue.columns = ['cust_id','revenue']

    return tx_revenue

In [ ]:
customer_revenue=calculate_revenue(transactions)
customer_revenue.head()

In [ ]:
customer_revenue.loc[customer_revenue['cust_id']=='0009655768c64bdeb2e877511632db8f']

In [ ]:
# create ordered clusters of the customer frequency
clusterd_customer_revenue = create_clusters(clusters_num=4, target_field_name='revenue', df=customer_revenue,ascending=True)
clusterd_customer_revenue.head()

In [ ]:
# determine the cluster for a particular customer_id
clusterd_customer_revenue.loc[clusterd_customer_revenue['cust_id']=='0009655768c64bdeb2e877511632db8f']

In [ ]:
# merge the dataframes
combined_rfm = pd.merge(clusterd_customer_revenue, clusterd_customer_recency, on='cust_id')
combined_rfm.head()

In [ ]:
combined_rfm=pd.merge(combined_rfm,clusterd_customer_fq,on='cust_id')
combined_rfm.head()

In [ ]:
combined_rfm['overall_score']=combined_rfm['recency_cluster']+combined_rfm['frequency_cluster']+combined_rfm['revenue_cluster']

In [ ]:
combined_rfm.head()

In [ ]:
## Get the overall score for the customer_segment

combined_rfm.groupby('overall_score')['recency','frequency','revenue'].mean()

In [ ]:
# create customer_segments

combined_rfm['RFM_segement'] = pd.qcut(combined_rfm['overall_score'], 3, labels=['Low-Value','Mid-Value', 'High-Value' ])
combined_rfm.head()

In [ ]:
#check a particular instances of cust_id

combined_rfm.loc[combined_rfm['cust_id']=='0009655768c64bdeb2e877511632db8f']

In [ ]:
combined_rfm.shape


In [ ]:
combined_rfm.info()

In [ ]:
combined_rfm.isna().sum()

In [ ]:
# final transaction dataframe
final_transactions_df = pd.merge(customers_transactions_df, combined_rfm, on='cust_id', how='inner')
final_transactions_df.head()

In [ ]:
final_transactions_df.shape

In [ ]:
final_transactions_df.info()

In [ ]:
# Checking is there any null value
final_transactions_df.isna().sum()

In [ ]:
profile_df.info()

In [ ]:
profile_df.isna().sum()

In [ ]:
# combine the cleaned profile with the final transactions data frame

offers_tx_profile_merged = final_transactions_df.merge(profile_df,on='cust_id', how='left')
offers_tx_profile_merged.tail(10)

In [ ]:
offers_tx_profile_merged.info()

In [ ]:
# fill nan with 0 for viewtime column
offers_tx_profile_merged['view_time'].fillna(0, inplace=True)
offers_tx_profile_merged.info()

In [ ]:
offers_tx_profile_merged.gender.value_counts()

In [ ]:
offers_tx_profile_merged.head()

In [ ]:
offers_tx_profile_merged.to_csv('analyzed_tx_offers_RFM_profile.csv', index = None, header=True)

In [ ]:
offers_tx_profile_merged.loc[offers_tx_profile_merged['cust_id']=='0610b486422d4921ae7d2bf64640c50b']

In [ ]:
offers_tx_profile_merged=offers_tx_profile_merged.dropna()

In [ ]:
offers_tx_profile_merged.reset_index(drop=True)

In [ ]:
categorical_columns_to_handle = ['gender', 'RFM_segement']
offers_tx_profile_merged_categorical = pd.get_dummies(data= offers_tx_profile_merged,columns=categorical_columns_to_handle)
offers_tx_profile_merged_categorical.head()

In [ ]:
# map the segment as follows: TR --> 0, CR --> 1, TN -->3, CN-->3
segment_dict = {'TR':0, 'CR':1, 'TN':2, 'CN':3}
offers_tx_profile_merged_categorical.segment.replace(segment_dict, inplace=True)
offers_tx_profile_merged_categorical.head()

In [ ]:
offers_tx_profile_merged_categorical.info()

In [ ]:
exploration_df=offers_tx_profile_merged.copy()

In [ ]:
exploration_df.info()

In [ ]:
exploration_df.gender.value_counts()

In [ ]:
exploration_df.RFM_segement.value_counts()

##**RFM Distribution**

In [ ]:

data=((exploration_df.RFM_segement.value_counts()/exploration_df.RFM_segement.value_counts().sum())* 100).round(1)

plt.figure(figsize = (10, 8))
yticklabels = [str(y) + '%' for y in np.arange(0,50,10)]
plt.yticks(np.arange(0,50,10), labels = yticklabels)
plt.xticks(rotation = 45, fontsize = 12)
plt.xlabel('RFM Groups', fontsize = 15)
plt.ylabel('Percentage %', fontsize = 15)
plt.title('RFM Groups Distribution', fontsize = 18)

bar_plot = plt.bar(x = data.index.values, height = data)
for i,bar in enumerate(bar_plot.patches):
    x, y = bar.get_xy()
    plt.text(x+bar.get_width()/2, y+bar.get_height()+0.2,
            str(data[i]) + '%',
            ha = 'center', weight = 'bold')

plt.tight_layout()
plt.show()

##**Customer groups distribution**

In [ ]:

data=((exploration_df.segment.value_counts()/exploration_df.segment.value_counts().sum())* 100).round(1)

plt.figure(figsize = (10, 8))
yticklabels = [str(y) + '%' for y in np.arange(0,50,10)]
plt.yticks(np.arange(0,50,10), labels = yticklabels)
plt.xticks(rotation = 45, fontsize = 12)
plt.xlabel('Customer Groups', fontsize = 15)
plt.ylabel('Percentage %', fontsize = 15)
plt.title('Segment Distribution', fontsize = 18)

bar_plot = plt.bar(x = data.index.values, height = data)
for i,bar in enumerate(bar_plot.patches):
    x, y = bar.get_xy()
    plt.text(x+bar.get_width()/2, y+bar.get_height()+0.2,
            str(data[i]) + '%',
            ha = 'center', weight = 'bold')

plt.tight_layout()
plt.show()

##**Summary**

- 40% of the customers are in the Treatment response group
- CR had 14.9% of the customers

In [ ]:
exploration_df.offer_alias.value_counts()

##**Demographics of RFM segments such as:**
- Age groups
- Income groups
- Gender

In [ ]:
plt.figure(figsize = [10, 5])
base_color = sns.color_palette()[0]
sns.boxplot(data = exploration_df, y = 'age', x = 'RFM_segement', color = base_color);
plt.title('RFM segment versus customer age ');
exploration_df.groupby('RFM_segement');

**the age is not a factor in the RFM segment it seems. though the RFM segment with low value seem to contain the smallest age group from 48 - 64 (quratile range)**

##**Purchasing behavior in RFM clusters:**
- Total Spent
- Profit
- Number of completed offers
- Segment (TR, TN, CR, CN)

In [ ]:
sns.boxplot(data = exploration_df, y = 'income', x = 'RFM_segement', color = base_color);

In [ ]:
# total spent vs RFM segment
sns.boxplot(data = exploration_df, y = 'total_spent', x = 'RFM_segement', color = base_color);
plt.title('Total amount spent vs RFM segment');
plt.ylim((-10,80));

In [ ]:
sns.boxplot(data = exploration_df, y = 'profit', x = 'RFM_segement', color = base_color);
plt.title('profit vs RFM segment');
plt.ylim((-10,80));

In [ ]:
# Customer segments vs RFM clusters
exploration_df.groupby(['RFM_segement', 'segment']).size() /exploration_df.groupby(['RFM_segement']).size()

In [ ]:
#  Segment (TR, TN, CR, CN) vs RFM
fig = plt.figure(figsize=(14,14))
fig.subplots_adjust(hspace=.3, wspace = 0.3)

plt.subplot(2, 2, 1)
ct_counts = exploration_df.groupby(['RFM_segement', 'segment']).size()
ct_counts = ct_counts.reset_index(name = 'count')
ct_counts = ct_counts.pivot(index = 'segment', columns = 'RFM_segement', values = 'count')
sns.heatmap(ct_counts, annot = True, fmt = 'g');

plt.subplot(2, 2, 2)
ct_counts = exploration_df.groupby(['RFM_segement', 'segment']).size()/exploration_df.groupby([ 'RFM_segement']).size()
ct_counts = ct_counts.reset_index(name = 'count')
ct_counts = ct_counts.pivot(index = 'segment', columns = 'RFM_segement', values = 'count')
sns.heatmap(ct_counts, annot = True, fmt = 'g');

plt.subplot(2, 2, 3)
ct_counts = exploration_df.groupby(['RFM_segement', 'segment']).size()/exploration_df.groupby([ 'segment']).size()
ct_counts = ct_counts.reset_index(name = 'count')
ct_counts = ct_counts.pivot(index = 'segment', columns = 'RFM_segement', values = 'count')
sns.heatmap(ct_counts, annot = True, fmt = 'g');

fig.suptitle('Customer Group vs RFM segment');

In terms of the segments the RFM clusters are distributed as follows:

- CN comprise of 56% of the Low value RFM clusters segments, followed by Mid value RFM clusters.
- CR mostly in the Mid Value segment with 38.5% followed by 33.7% in the high value RFM cluster.
- TN comprise of 62% low value RFM clusters. and below 10% from the high value RFM cluster.
- TR: highest portion comes from the mid value RFM cluster with 40% followed by high value with 35.6%

Compring with respect to the RFM cluster:

-  Low value have 42% in the TN segment and similar percentages for CN and TR.
- Mid Value RFM cluster comprise of 40% for the TR segment which is double the other RFM clusters.
- High Value comprise of 60% of TR then 44% of Mid value Customers.

The above observations make sense since the high-value customers are the ones who are most frequent, spent the most and had the most recent interactions. This indicates that high and mid-value customers are responsive to targeted marketing while it did not make a difference in the Low values.

Next, I will compared the number of compelted offers across different segments.

In [ ]:
exploration_df['num_completed_offers'] = exploration_df['num_informational_completed'] + exploration_df['num_discount_completed'] + exploration_df['num_bogo_completed']

In [ ]:
fig = plt.figure(figsize=(10,10))
fig.subplots_adjust(hspace=.3, wspace = 0.3)

plt.subplot(2, 2, 1)
sns.boxplot(data = exploration_df, y = 'num_bogo_completed', x = 'RFM_segement', color = base_color);
plt.title('num of completed BOGO offers vs RFM segment');

plt.subplot(2, 2, 2)
sns.boxplot(data = exploration_df, y = 'num_discount_completed', x = 'RFM_segement', color = base_color);
plt.title('num of completed Discount offers vs RFM segment');

plt.subplot(2, 2, 3)
sns.boxplot(data = exploration_df, y = 'num_informational_completed', x = 'RFM_segement', color = base_color);
plt.title('num of completed informational offers vs RFM segment');


plt.subplot(2, 2, 4)
sns.boxplot(data = exploration_df, y = 'num_completed_offers', x = 'RFM_segement', color = base_color);
plt.title('Num of completed_offers vs RFM segment');

fig.suptitle('Number of completed offers vs RFM segment');

**As expected the number of completed offers for the High value Cluster is the highest compared followed by mid value and RFM clusters.**

##**Customer group vs demographic**

In [ ]:
# income vs segment
sns.boxplot(data = exploration_df, y = 'income', x = 'segment', color = base_color);
plt.title('income vs customer segment');

In [ ]:
# total spent vs customer segment
sns.boxplot(data = exploration_df, y = 'total_spent', x = 'segment', color = base_color);
plt.title('income vs customer segment');
plt.ylim((-10,80));

**The difference in the amount of money spent between the treatment and control groups is significant. The highest spent value amount of value spent in TN and CN groups are less than the lower limit of the TR and CR groups**

In [ ]:
ct_counts = exploration_df.groupby(['gender', 'segment']).size()/exploration_df.groupby([ 'gender']).size()
ct_counts = ct_counts.reset_index(name = 'count')
ct_counts = ct_counts.pivot(index = 'segment', columns = 'gender', values = 'count')
sns.heatmap(ct_counts, annot = True, fmt = 'g');

**We Can observe that the highest percentage of Others and females is in the TR group. the Males have equal percentages of TN groups.**

In [ ]:

# total spent vs RFM vs cluster -->
fig = plt.figure(figsize=(10,10));
g = sns.FacetGrid(data = exploration_df, col='segment', col_wrap=2, height =4);
(g.map(sns.boxplot, 'RFM_segement', 'total_spent')).fig.subplots_adjust(wspace=.35, hspace=.35);
plt.ylim((-10,80));
plt.subplots_adjust(top=0.9);
g.fig.suptitle('Total spent vs Segment vs RFM segment', fontsize = 16);

**We can conclude that the TR and CR have the highest amount spent with the high value of RFM cluster having the highest amount spent. and they have similar range of money spent and this can be observed in the CN and TN groups**

In [ ]:
# Age vs RFM vs cluster -->
fig = plt.figure(figsize=(10,10));
g = sns.FacetGrid(data = exploration_df, col='segment', col_wrap=2, height =4);
(g.map(sns.boxplot, 'RFM_segement', 'age')).fig.subplots_adjust(wspace=.35, hspace=.35);
plt.subplots_adjust(top=0.9);
g.fig.suptitle('Age vs Segment vs RFM segment', fontsize = 16);

**The age have similar trends across different customer segments. with the high valued RFM cluster having a wider range of age covered and the response groups both have higher age.**

In [ ]:
# income vs RFM vs cluster -->
fig = plt.figure(figsize=(10,10));
g = sns.FacetGrid(data = exploration_df, col='segment', col_wrap=2, height =4);
(g.map(sns.boxplot, 'RFM_segement', 'income')).fig.subplots_adjust(wspace=.35, hspace=.35);
plt.subplots_adjust(top=0.9);
g.fig.suptitle('Income vs Segment vs RFM segment', fontsize = 16);

- The highest income can be found with the customers from the response groups such as TR and CR group.
- The highest income ranges is for CR customer group
lowest income range is for CN group